In [ ]:
#### Working with workspace and client
# details
# pip install azureml-mlflow
subscritpion_id="53eb2592-79d7-4d14-a92a-b97966ccf1c9"
resource_group="bdc"
workspace="ey2025"

In [2]:
# authenticate
from azure.identity import InteractiveBrowserCredential
credential = InteractiveBrowserCredential(tenant_id="13a86542-2185-4187-8e07-7512f5525c55")

# create ML client
from azure.ai.ml import MLClient
ml_client = MLClient(credential=credential,subscription_id=subscritpion_id,
                     resource_group_name=resource_group,workspace_name=workspace)


In [3]:
# access the workspace
from azure.ai.ml.entities import Workspace
try:
    ws = ml_client.workspaces.get(workspace)
    print("Workspace already exists")
except:
    print("Creating new workspace")
    ws = Workspace(name=workspace,location='centralindia')
    ws = ml_client.workspaces.begin_create(ws)

Workspace already exists


In [4]:
ws.mlflow_tracking_uri

'azureml://eastasia.api.azureml.ms/mlflow/v2.0/subscriptions/53eb2592-79d7-4d14-a92a-b97966ccf1c9/resourceGroups/bdc/providers/Microsoft.MachineLearningServices/workspaces/ey2025'

In [5]:
import pandas as pd
import numpy as np, matplotlib.pyplot as plt, mlflow, mlflow.sklearn, os
mlflow.set_tracking_uri(ws.mlflow_tracking_uri)
mlflow.set_experiment("Banking_Customer_churn")

/home/labuser/anaconda3/lib/python3.12/site-packages/azureml/mlflow/_protos/aml_service_pb2.py:10: UserWarning: google.protobuf.service module is deprecated. RPC implementations should provide code generator plugins which generate code specific to the RPC implementation. service.py will be removed in Jan 2025
  from google.protobuf import service as _service
2025/03/05 05:12:13 INFO mlflow.tracking.fluent: Experiment with name 'Banking_Customer_churn' does not exist. Creating a new experiment.


<Experiment: artifact_location='', creation_time=1741151533056, experiment_id='d56e240b-9e67-4c71-ab2e-0d8a77f8e744', last_update_time=None, lifecycle_stage='active', name='Banking_Customer_churn', tags={}>

In [6]:
# data processing
url = "https://raw.githubusercontent.com/anshupandey/Machine_Learning_Training/refs/heads/master/datasets/Bank_churn_modelling.csv"
df = pd.read_csv(url)
x = df[['CreditScore', 'Geography', 'Gender', 'Age', 'Balance', 'NumOfProducts', 'IsActiveMember']]
y =df['Exited']
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(x,y,test_size=0.2,stratify=y,random_state=5)
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
transformer = ColumnTransformer([('ohe',OneHotEncoder(drop="first"),[1,2]),],remainder='passthrough')

In [11]:
eval_data = xtest.copy()
eval_data['label'] = ytest

In [15]:
from sklearn import metrics
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import Pipeline
mlflow.sklearn.autolog()

with mlflow.start_run():
    model_pipeline = Pipeline([("transformer",transformer),
                               ("model",DecisionTreeClassifier(criterion='gini',min_samples_leaf=40,
                                                               max_depth=8, class_weight='balanced',random_state=5))])
    model_pipeline.fit(xtrain,ytrain)

    model_uri = mlflow.get_artifact_uri("model")

    #model evaluation
    #result = mlflow.evaluate(model=model_uri,data=eval_data,targets='label',model_type='classifier',evaluators=['default'],
                             #evaluator_config={"default":{"metric_prefix":"test_",}})

2025/03/05 06:14:37 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/home/labuser/anaconda3/lib/python3.12/site-packages/mlflow/types/utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details."
2025/03/05 06:14:39 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/home/labuser/anaconda3/lib/python3.12/site-p

🏃 View run helpful_spider_1r299pwf at: https://eastasia.api.azureml.ms/mlflow/v2.0/subscriptions/53eb2592-79d7-4d14-a92a-b97966ccf1c9/resourceGroups/bdc/providers/Microsoft.MachineLearningServices/workspaces/ey2025/#/experiments/d56e240b-9e67-4c71-ab2e-0d8a77f8e744/runs/0923ee90-28c3-48e9-be71-9d717b523adf
🧪 View experiment at: https://eastasia.api.azureml.ms/mlflow/v2.0/subscriptions/53eb2592-79d7-4d14-a92a-b97966ccf1c9/resourceGroups/bdc/providers/Microsoft.MachineLearningServices/workspaces/ey2025/#/experiments/d56e240b-9e67-4c71-ab2e-0d8a77f8e744


In [17]:
# access the last run
last_run = mlflow.last_active_run()
runid = last_run.info.run_id

#model registration
model_uri = f"runs:/{runid}/model"
model_name = "churn_Pred"
mlflow.register_model(model_uri,model_name)

Registered model 'churn_Pred' already exists. Creating a new version of this model...
2025/03/05 06:18:29 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: churn_Pred, version 1
Created version '1' of model 'churn_Pred'.


<ModelVersion: aliases=[], creation_timestamp=1741155509662, current_stage='None', description=None, last_updated_timestamp=1741155509662, name='churn_Pred', run_id='0923ee90-28c3-48e9-be71-9d717b523adf', run_link='', source='azureml://eastasia.api.azureml.ms/mlflow/v2.0/subscriptions/53eb2592-79d7-4d14-a92a-b97966ccf1c9/resourceGroups/bdc/providers/Microsoft.MachineLearningServices/workspaces/ey2025/experiments/d56e240b-9e67-4c71-ab2e-0d8a77f8e744/runs/0923ee90-28c3-48e9-be71-9d717b523adf/artifacts/model', status='READY', status_message=None, tags={}, user_id='', version='1'>

## DEployment over ACI

In [26]:
%%writefile AzureML/aciconfig.json

{
    "computeType":"aci",
    "containerResourceRequirement":
    {
        "cpu":2,
        "memoryInGB":4
    },
    "location":"eastasia",

}

Overwriting AzureML/aciconfig.json


In [27]:
import json
from mlflow.deployments import get_deploy_client
client = get_deploy_client(ws.mlflow_tracking_uri)

# deployment configuraiton
deploy_config_path = 'AzureML/aciconfig.json'
config = {"deploy-config-file":deploy_config_path}
model_version = 1

In [28]:
webservice = client.create_deployment(model_uri=f"models:/{model_name}/{model_version}",
                                      config=config,name='churn-aci')

/home/labuser/anaconda3/lib/python3.12/site-packages/azureml/mlflow/deploy/_util.py:64: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_model_version_stages`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  if model_stage_or_version in client.get_model_version_stages(None, None):


Running
2025-03-05 06:58:11+00:00 Creating Container Registry if not exists.
2025-03-05 06:58:13+00:00 Use the existing image.
2025-03-05 06:58:13+00:00 Generating deployment configuration.
2025-03-05 06:58:17+00:00 Submitting deployment to compute.
2025-03-05 06:58:31+00:00 Checking the status of deployment churn-aci....
2025-03-05 06:59:44+00:00 Checking the status of inference endpoint churn-aci..
Failed


Unable to retrieve continuation token: Polling method 'MMSPolling' doesn't support get_continuation_token


test


MlflowException: Error while creating deployment